In [2]:
from glob import glob
import re, csv
from zipfile import ZipFile, BadZipFile
from gzip import GzipFile
from tarfile import TarFile, ReadError
from io import BytesIO
import os

In [3]:
files = ['/home/fekete/Dropbox (CEU Econ)/REStud papers/{}'.format(i.strip()) for i in open('list.csv').readlines()]

In [4]:
len(files)

311

In [5]:
MS_ID = re.compile(r'[0-9]{4,5}')
ZIP_FILE = re.compile(r'\.zip$', flags=re.IGNORECASE)
TAR_FILE = re.compile(r'\.tar', flags=re.IGNORECASE)

In [6]:
NR_EXTENSIONS = { "ms_number": 0,"Python":0, "Stata":0, "R":0, "MatLab":0, "C":0, "Fortran":0, "Bash":0, "LaTeX": 0, "C++": 0, "SAS":0}
#store number of used extensions

In [7]:
EXTENSIONS = {"ms_number": "0", "Python": ".py", "Stata":".do", "R": ".R", "MatLab": ".m", "C": ".c", "Fortran": re.compile(r'[.][f][0-9]{2}'), "Bash": ".sh", "LaTeX": ".tex", "C++": ".cpp", "SAS": ".sas"}
#extension ids

In [8]:
MANIFESTO = {"ms_number": 0, "file_name": 0, "file_type": 0, "file_size": 0}
#dictionary to help writing in csv

In [9]:
def count_extensions(name, extensions, nr_extensions):
    ext=os.path.splitext(name)[1]
    for key in nr_extensions:
        if key == "Fortran" and extensions["Fortran"].search(ext) is not None:
            nr_extensions[key]+=1
        elif ext == extensions[key] :
            nr_extensions[key]+=1
            
#stock number of extensions

def reset_extension_nr(extensions):
    for key in extensions:
        extensions[key]=0
        
def manifesto_maker(file, name, manifesto_dict):
    
    manifesto_dict["file_name"]=os.path.split(name)[1]
    manifesto_dict["file_type"]=os.path.splitext(name)[1]
    if type(file)==ZipFile:
        manifesto_dict["file_size"]=file.getinfo(name).file_size
    elif type(file)==TarFile:
        manifesto_dict["file_size"]=file.getmember(name).size
        #write filename, extesion and file size of files in the zip into dictionary

In [15]:
def tree_in_zip(path, stream=None):
    print('Trying {}'.format(path))
    if stream is not None and ZIP_FILE.search(path) is not None:
        zf = ZipFile(stream)
        #Check if the file is not the first in the loop and wheter it is a .zip.

    elif stream is None and ZIP_FILE.search(path) is not None:
        zf = ZipFile(path)
        #Check if the first file is .zip.
    else:
        if stream is not None and TAR_FILE.search(path) is not None :
            try: 
                zf=TarFile.open(fileobj=stream, encoding="utf-8",mode='r:*')
            except ReadError:
                pass
            #If the file is not the first in the loop and it is a .tar(.xx) tries to read it as an archive.
        elif stream is None and TAR_FILE.search(path) is not None:
            try:
                zf=TarFile.open(path, mode='r:*')
            except ReadError:
                pass
            #If the first file is a .tar(.xx) tries to read it as an archive.
    try:
        
        if type(zf)==ZipFile:
            try:
                for name in zf.namelist(): 
                    count_extensions(name, EXTENSIONS, NR_EXTENSIONS)
                    #Count extensions of interest in the archive.
                    manifesto_maker(zf, name, MANIFESTO)
                    #fills in the dictionary
                    if ZIP_FILE.search(name) is not None:
                        print('-Found {}'.format(name))
                        yield from tree_in_zip(path+'/'+name, BytesIO(zf.read(name)))
                    elif TAR_FILE.search(name) is not None:
                        print('-Found {}'.format(name))
                        yield from tree_in_zip(path+'/'+name, BytesIO(zf.read(name)))
                    else:
                        yield path+'/'+name
            except BadZipFile:
                pass
            #Search for further archives (.zip/.tar). Exception needed to not to stop at a corrupted archive.
        elif type(zf)==TarFile:
            #No need for try checked the file at the begining.
                for name in zf.getnames():
                    count_extensions(name, EXTENSIONS, NR_EXTENSIONS)
                    manifesto_maker(zf, name, MANIFESTO)
                    if ZIP_FILE.search(name) is not None:
                        print('-Found {}'.format(name))
                        yield from tree_in_zip(path+'/'+name, BytesIO(zf.read(name)))
                    elif TAR_FILE.search(name) is not None:
                        print('- Found {}'.format(name))
                        yield from tree_in_zip(path+'/'+name, BytesIO(zf.read(name)))    
                    else:
                        yield path+'/'+name
        else:
            pass
        #if file is not .tar/.zip skip it
    except UnboundLocalError:
        pass
    #Because of the .tar checks at the begining could happen that there's no zf defined. This way the loop goes on if found a corrupted archive.    except UnboundLocalError:
    except MemoryError:
        pass


In [19]:
ms_language = csv.DictWriter(open('MS_language.csv', 'w'), fieldnames=list(NR_EXTENSIONS.keys()))
ms_language.writeheader()
ms_files = csv.DictWriter(open('MS_manifesto.csv', 'w'), fieldnames=list(MANIFESTO.keys()))
ms_files.writeheader()
#creat 2 separate csvs for the extensions and manifesto

In [20]:
i = 0
for name in files:
    i = i + 1
    print(i)
    reset_extension_nr(MANIFESTO)
    reset_extension_nr(NR_EXTENSIONS)
    #Must reset counting outside the functions, because we write in this loop.
    match = MS_ID.search(name)
    if match:
        for entry in tree_in_zip(name):
            MANIFESTO["ms_number"] = match.group()
            ms_files.writerow(MANIFESTO)
        NR_EXTENSIONS["ms_number"] = match.group()
        ms_language.writerow(NR_EXTENSIONS)

1
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./21211 Caliendo/Supplementary.zip
2
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./20916 Lise/Supplementary.zip
3
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./22911 Chassang/Supplementary.zip
4
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./19047 Llull/Supplementary.zip
-Found Supplementary/Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./19047 Llull/Supplementary.zip/Supplementary/Datafortheweb.zip
5
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./23266 Piccolo/Supplementary.zip
6
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./21451 Gowrisankaran/supplementary_materials.zip
7
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./16414 Xu/Supplementary.zip
8
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./19562 Freyberger/Supplementary.zip
-Found Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./19562 Freyberger/Supplementary.zip/Datafo

Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./20276 Marcet/Supplementary.zip/supplementary/finalcodesforsubmissionlongandshort.zip/finalcodesforsubmissionlongandshort/numericalcodes/shocks.zip
-Found finalcodesforsubmissionlongandshort/numericalcodes/simulationsTable/simulations3bonds.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./20276 Marcet/Supplementary.zip/supplementary/finalcodesforsubmissionlongandshort.zip/finalcodesforsubmissionlongandshort/numericalcodes/simulationsTable/simulations3bonds.zip
-Found finalcodesforsubmissionlongandshort/numericalcodes/simulationsTable/simulationsBBlending.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./20276 Marcet/Supplementary.zip/supplementary/finalcodesforsubmissionlongandshort.zip/finalcodesforsubmissionlongandshort/numericalcodes/simulationsTable/simulationsBBlending.zip
-Found finalcodesforsubmissionlongandshort/numericalcodes/simulationsTable/simulationsBBnolending.zip
Trying /home/fekete/Dropbox (CEU Eco

-Found Supplementary/Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./24938 Acemoglu/Supplementary.zip/Supplementary/Datafortheweb.zip
50
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./21959 Tirole/Supplementary.zip
51
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./23024 Holden/Supplementary.zip
52
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./24175 Rosenzweig/Supplementary.zip
53
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./24360 Igami/02_Supplementary.zip
54
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./18706 Duranton/Supplementary.zip
-Found Supplementary/Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./18706 Duranton/Supplementary.zip/Supplementary/Datafortheweb.zip
55
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./22132 Aguirregabiria/Old/Supplementary.zip
56
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./22132 Aguirregabiria/Supplementary (1).zip
57
Trying /home/fekete/Drop

89
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./23113 Andersson/Supplamentary.zip
90
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./24530 Bursztyn/supplementary.zip
91
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./22604 Le Barbanchon/Supplementary.zip
-Found Supplementary/Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./22604 Le Barbanchon/Supplementary.zip/Supplementary/Datafortheweb.zip
92
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./18592 MeyerTerVehn/Supplementary.zip
93
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./26123 Tabellini/MS26123_-_corrected_and_updated_material/Supplementary.zip
94
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./25576 Bonatti/supplementary.zip
95
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./23129 Gavazza/Supplementary.zip
96
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./24631 Stroebel/Supplementary.zip
97
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./

Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 2/16381 Anderson/Supplementary.zip/Datafortheweb.zip
-Found NHTSA_Crash_Test.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 2/16381 Anderson/Supplementary.zip/Datafortheweb.zip/NHTSA_Crash_Test.zip
140
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 2/16886 Gopinath/Supplementary.zip
-Found Supplementary/Code for simulation replication.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 2/16886 Gopinath/Supplementary.zip/Supplementary/Code for simulation replication.zip
141
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 3/15722 Rudanko/Supplementary.zip
142
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 3/16154 Lewis/supplementary.zip
143
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/81/81 3/15459 Fleurbaey/Suppleme

189
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/17443 Caliendo/Supplementary.zip
190
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/16334 Rhodes/Supplementary.zip
191
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/16238 Georgiadis/Supplementary.zip
192
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/16119 Grossman/Supplementary.zip
193
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/16482 Rappoport/Supplementary.zip
-Found Codes and Data.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/16482 Rappoport/Supplementary.zip/Codes and Data.zip
194
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/17679 Leibbrandt/Supplementary.zip
195
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/82/82 1/15503 Gandhi/Supplementary.zip
196
T

Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/84/84 2/19262 Kopecky/Supplementary.zip/Supplementary/HRS_DATA_WORK.zip
-Found Supplementary/QUANTITATIVE_MODEL.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/84/84 2/19262 Kopecky/Supplementary.zip/Supplementary/QUANTITATIVE_MODEL.zip
-Found Supplementary/TWO_PERIOD_MODEL.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/84/84 2/19262 Kopecky/Supplementary.zip/Supplementary/TWO_PERIOD_MODEL.zip
242
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/84/84 2/19281 Fort/Supplementary.zip
-Found Supplementary/Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/84/84 2/19281 Fort/Supplementary.zip/Supplementary/Datafortheweb.zip
243
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/84/84 2/19998 Lindenlaub/Supplementary.zip
-Found Datafortheweb.zip
Trying /home/fekete/Dropb

Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/85 2/20281 Kredler/Supplementary.zip/Datafortheweb.zip
-Found __MACOSX/._Datafortheweb.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/85 2/20281 Kredler/Supplementary.zip/__MACOSX/._Datafortheweb.zip
305
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/85 2/20551 Varela/Supplementary.zip
-Found Supplementary/datacodes.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/85 2/20551 Varela/Supplementary.zip/Supplementary/datacodes.zip
-Found __MACOSX/Supplementary/._datacodes.zip
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/85 2/20551 Varela/Supplementary.zip/__MACOSX/Supplementary/._datacodes.zip
306
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./Ready at Publisher/85 2/22879 DellaVigna/supplementary.zip
307
Trying /home/fekete/Dropbox (CEU Econ)/REStud papers/./20437 Halevy/Supplementary.zip
-Foun